# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather_df = pd.read_csv('../WeatherPy/Output Data/City_Data.csv')
del cities_weather_df['Unnamed: 0']
cities_weather_df.head(3)

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,gat,IL,31.61,34.76,77.0,62,0,2.73
1,hilo,US,19.73,-155.09,84.2,66,40,10.29
2,georgetown,MY,5.41,100.34,77.0,94,20,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in coordinates
coordinates = cities_weather_df[['Latitude', 'Longitude']]
map_weight = cities_weather_df['Humidity (%)']

In [11]:
# Plot Heatmap - Courtesy of tutor(Sandhya Kumari)'s help in figuring out size
fig = gmaps.figure(center = (20,0), zoom_level=2.2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=map_weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
clouds = cities_weather_df['Cloudiness (%)']
temp = cities_weather_df['Max Temperature (F)']
wind = cities_weather_df['Wind Speed (mph)']
humidity = cities_weather_df['Humidity (%)']

In [24]:
# My kind of weather Condition: 
## Maximum temperature: 70-75
## Cloudiness <30%
## Humidity <70%
## Wind <10

ideal_weather_df = cities_weather_df.loc[(clouds < 20) & (humidity < 70) & (temp <= 75) & (temp >= 70) & (wind < 10), :]
ideal_weather_df
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number like 10.
ideal_weather_df = ideal_weather_df.dropna() #nothing to drop...
ideal_weather_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
70,zhangye,CN,38.93,100.45,70.52,36,0,2.51
112,jiuquan,CN,39.74,98.52,70.90,37,0,2.86
202,jinchang,CN,38.50,102.17,72.46,31,9,4.36
368,lodja,CD,-3.48,23.43,72.00,65,7,1.81
378,varzea da palma,BR,-17.60,-44.73,70.88,53,6,6.33
424,kamarion,GR,36.80,25.82,75.00,65,0,1.99
542,kinkala,CG,-4.36,14.76,73.29,53,19,6.60
548,dustlik,UZ,40.52,68.04,72.59,40,11,2.93
556,alasehir,TR,38.35,28.52,71.01,44,0,5.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
# Reset the index
hotel_df = ideal_weather_df.reset_index(drop=True)
print(len(hotel_df))
hotel_df

9


,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,zhangye,CN,38.93,100.45,70.52,36,0,2.51
1,jiuquan,CN,39.74,98.52,70.90,37,0,2.86
2,jinchang,CN,38.50,102.17,72.46,31,9,4.36
3,lodja,CD,-3.48,23.43,72.00,65,7,1.81
4,varzea da palma,BR,-17.60,-44.73,70.88,53,6,6.33
5,kamarion,GR,36.80,25.82,75.00,65,0,1.99
6,kinkala,CG,-4.36,14.76,73.29,53,19,6.60
7,dustlik,UZ,40.52,68.04,72.59,40,11,2.93
8,alasehir,TR,38.35,28.52,71.01,44,0,5.99


In [41]:
# Add hotel name column in the hotel dataframe
hotel_df['Hotel Name'] = ''

In [46]:
from pprint import pprint
# Store latitude and longitude in hotel_locations
hotel_locations = hotel_df[['Latitude', 'Longitude']]

# Use the Google Places API to find the first hotel for each city located within 5000 meters of tne coordinates.
radius = 5000

# Set up a parameters dictionary for gmap API
myparams = {
    "keyword": 'hotel',
    "radius": radius,
    "key": g_key
}

hotel_list = []

for index, row in hotel_df.iterrows():
    # hotel url
    hotelbase_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row['Latitude']
    lng = row['Longitude']
    
    # update address key value in dictionary
    myparams['location'] = f'{lat}, {lng}'
    
    # make API request
    response = requests.get(hotelbase_url, params=myparams)
    
    # convert response to json
    hotel_data = response.json()
   
    try:
        # Add hotel name to the list
        hotel_list.append(hotel_data['results'][0]['name'])
        print(f"{hotel_data['results'][0]['name']} found.")
    
    except:
        # If hotel doesn't exist, put 'N/A' to the list
        hotel_list.append('N/A')
        print(f"No hotel found in {row['City']}.")
        pass

    
hotel_df['Hotel Name'] = hotel_list

Jinyang International Hotel found.
Jiuquan Hotel found.
Jinchang Hotel found.
No hotel found in lodja.
Hotel Casa Grande e Restaurante Forno e Fogão found.
Amorgion Hotel found.
Hotel la Pepeche Kinkala found.
No hotel found in dustlik.
CAN DELUXE HOTEL found.


In [47]:
# Display hotel_df dataframe
hotel_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
0,zhangye,CN,38.93,100.45,70.52,36,0,2.51,Jinyang International Hotel
1,jiuquan,CN,39.74,98.52,70.90,37,0,2.86,Jiuquan Hotel
2,jinchang,CN,38.50,102.17,72.46,31,9,4.36,Jinchang Hotel
3,lodja,CD,-3.48,23.43,72.00,65,7,1.81,N/A
4,varzea da palma,BR,-17.60,-44.73,70.88,53,6,6.33,Hotel Casa Grande e Restaurante Forno e Fogão
5,kamarion,GR,36.80,25.82,75.00,65,0,1.99,Amorgion Hotel
6,kinkala,CG,-4.36,14.76,73.29,53,19,6.60,Hotel la Pepeche Kinkala
7,dustlik,UZ,40.52,68.04,72.59,40,11,2.93,N/A
8,alasehir,TR,38.35,28.52,71.01,44,0,5.99,CAN DELUXE HOTEL


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [51]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info) 

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))